In [ ]:
#Install required libraries
!pip install opencv-python matplotlib numpy pillow scikit-learn


To build a dataset of office items, videos of each item from multiple angles were recorded and stored in the folder folders. This ensures the model sees items under different orientations and lighting conditions, which improves classification performance. A python script automatically extracts frames from each video. The script saves the frames as images in separate folders for each class.

In [ ]:
import cv2
import os

# Define classes
classes = ["scissor", "glueStick"]

# Base directories
VIDEO_BASE_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\videos"     
OUTPUT_BASE_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\raw"     

# Loop over classes
for cls in classes:
    video_dir = os.path.join(VIDEO_BASE_DIR, cls)
    output_dir = os.path.join(OUTPUT_BASE_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    for video_file in os.listdir(video_dir):
       

        cap = cv2.VideoCapture(os.path.join(video_dir, video_file))
        count = 0

        while True and count < 50:  # Limit to first 50 frames
            ret, frame = cap.read()
            if not ret:
                break
            frame_name = f"{os.path.splitext(video_file)[0]}_frame{count:04d}.jpg"
            cv2.imwrite(os.path.join(output_dir, frame_name), frame)
            count += 1

        cap.release()
        print(f"Extracted {count} frames from {cls}/{video_file}")

print("Done! Frames saved in:", OUTPUT_BASE_DIR)


Extracted 50 frames from scissor/WhatsApp Video 2025-10-16 at 14.27.15_5e69e809.mp4
Extracted 50 frames from scissor/WhatsApp Video 2025-10-16 at 14.27.15_71c3fcee.mp4
Extracted 50 frames from glueStick/IMG_3257.mov
Extracted 50 frames from glueStick/IMG_3259.mov
Extracted 50 frames from glueStick/IMG_3260.mov
Extracted 50 frames from glueStick/IMG_3267.mov
Extracted 50 frames from glueStick/IMG_3268.mov
Extracted 50 frames from glueStick/IMG_3275.mov
Done! Frames saved in: C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\raw


All images were renamed using their class prefix (e.g., pen00001.jpg) to keep filenames unique, organized, and easily identifiable by class.

In [7]:
# Renaming images in the folders
import os

# Root folder containing your raw image class folders
root_folder = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\raw"

# Allowed image extensions
image_extensions = ('.jpg', '.jpeg', '.png')

# Loop through each subfolder 
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    
    if os.path.isdir(folder_path):
        print(f"\nRenaming images in folder: {folder_name}")
        
        files = os.listdir(folder_path)
        
        # Filter only image files
        images = [f for f in files if f.lower().endswith(image_extensions)]
        images.sort()

        # Rename images in this folder with folder name prefix
        for index, filename in enumerate(images, start=1):
            new_name = f"{folder_name}{index:06}.jpg" 
            old_path = os.path.join(folder_path, filename)
            new_path = os.path.join(folder_path, new_name)

            try:
                os.rename(old_path, new_path)
                print(f"Renamed: {filename} -> {new_name}")
            except Exception as e:
                print(f"Error renaming {filename}: {e}")

print("\nRenaming completed successfully for all folders!")



Renaming images in folder: eraser
Renamed: eraser000001.jpg -> eraser000001.jpg
Renamed: eraser000002.jpg -> eraser000002.jpg
Renamed: eraser000003.jpg -> eraser000003.jpg
Renamed: eraser000004.jpg -> eraser000004.jpg
Renamed: eraser000005.jpg -> eraser000005.jpg
Renamed: eraser000006.jpg -> eraser000006.jpg
Renamed: eraser000007.jpg -> eraser000007.jpg
Renamed: eraser000008.jpg -> eraser000008.jpg
Renamed: eraser000009.jpg -> eraser000009.jpg
Renamed: eraser000010.jpg -> eraser000010.jpg
Renamed: eraser000011.jpg -> eraser000011.jpg
Renamed: eraser000012.jpg -> eraser000012.jpg
Renamed: eraser000013.jpg -> eraser000013.jpg
Renamed: eraser000014.jpg -> eraser000014.jpg
Renamed: eraser000015.jpg -> eraser000015.jpg
Renamed: eraser000016.jpg -> eraser000016.jpg
Renamed: eraser000017.jpg -> eraser000017.jpg
Renamed: eraser000018.jpg -> eraser000018.jpg
Renamed: eraser000019.jpg -> eraser000019.jpg
Renamed: eraser000020.jpg -> eraser000020.jpg
Renamed: eraser000021.jpg -> eraser000021.jpg

Renamed: eraser000502.jpg -> eraser000502.jpg
Renamed: eraser000503.jpg -> eraser000503.jpg
Renamed: eraser000504.jpg -> eraser000504.jpg
Renamed: eraser000505.jpg -> eraser000505.jpg
Renamed: eraser000506.jpg -> eraser000506.jpg
Renamed: eraser000507.jpg -> eraser000507.jpg
Renamed: eraser000508.jpg -> eraser000508.jpg
Renamed: eraser000509.jpg -> eraser000509.jpg
Renamed: eraser000510.jpg -> eraser000510.jpg
Renamed: eraser000511.jpg -> eraser000511.jpg
Renamed: eraser000512.jpg -> eraser000512.jpg
Renamed: eraser000513.jpg -> eraser000513.jpg
Renamed: eraser000514.jpg -> eraser000514.jpg
Renamed: eraser000515.jpg -> eraser000515.jpg
Renamed: eraser000516.jpg -> eraser000516.jpg
Renamed: eraser000517.jpg -> eraser000517.jpg
Renamed: eraser000518.jpg -> eraser000518.jpg
Renamed: eraser000519.jpg -> eraser000519.jpg
Renamed: eraser000520.jpg -> eraser000520.jpg
Renamed: eraser000521.jpg -> eraser000521.jpg
Renamed: eraser000522.jpg -> eraser000522.jpg
Renamed: eraser000523.jpg -> erase

After renaming and organising images, the number of images in each class folder was checked to determine if data augmentation was needed and to identify classes with fewer images.

In [8]:
import os

base_dir = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\raw"

classes = ["waterBottle", "pen", "mouse","eraser","glueStick","scissor","stapler","pencilBox","pencilSharpener","paperClip"]

for cls in classes:
    folder = os.path.join(base_dir, cls)
    if os.path.exists(folder):
        print(f"\nClass: {cls}")
        print("Number of images:", len(os.listdir(folder)))
        print("Example files:", os.listdir(folder)[:5])
    else:
        print(f"\n⚠ Folder not found: {folder}")



Class: waterBottle
Number of images: 1192
Example files: ['waterBottle000001.jpg', 'waterBottle000002.jpg', 'waterBottle000003.jpg', 'waterBottle000004.jpg', 'waterBottle000005.jpg']

Class: pen
Number of images: 2334
Example files: ['pen000001.jpg', 'pen000002.jpg', 'pen000003.jpg', 'pen000004.jpg', 'pen000005.jpg']

Class: mouse
Number of images: 1743
Example files: ['03DS76ZT7GVB_jpg.rf.14eb8aa6a4b0122bc090a222940f13a2.jpg', '08WBAV853XST_jpg.rf.bb34f831a36525e257e7b7c55897600d.jpg', '0BWP88MBBQWB_jpg.rf.989b354cfb00b971c586dcf0fa662407.jpg', '0DVLKXZHYAVT_jpg.rf.39af65a09d5292e30739b16ff24b9a9a.jpg', '0FLI0LRQ7MZI_jpg.rf.ffacbf7e1685886decdd0a44848f7c41.jpg']

Class: eraser
Number of images: 1375
Example files: ['eraser000001.jpg', 'eraser000002.jpg', 'eraser000003.jpg', 'eraser000004.jpg', 'eraser000005.jpg']

Class: glueStick
Number of images: 735
Example files: ['glueStick000001.jpg', 'glueStick000002.jpg', 'glueStick000003.jpg', 'glueStick000004.jpg', 'glueStick000005.jpg']

C

Image Resizing

Since some images have been taken from the internet, and others were captured from mobile phone camera, the images are of different sizes. Therefore, resizing is important before training a machine learning model as it makes the dataset uniform, memory-efficient, and compatible with the model architecture, which is crucial for effective training.

224×224 is chosen because it is a good compromise between preserving object detail and computational efficiency, and it matches the input size of most pre-trained models used for transfer learning.

In [ ]:
#Image Processing and cleaning
import os
import cv2

# Define paths
RAW_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\raw"
PROCESSED_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\processed"

#Desired Image size
imageSize=(224,224)

# Create processed directory if not exists
if not os.path.exists(PROCESSED_DIR):
    os.makedirs(PROCESSED_DIR)

#Loop through each class folder
for class_name in os.listdir(RAW_DIR):
    class_path = os.path.join(RAW_DIR, class_name)

    # Skip non-folders
    if not os.path.isdir(class_path):
        continue

    print(f"Processing class: {class_name}")

    # Create mirrored folder in processed/
    processed_class_path = os.path.join(PROCESSED_DIR, class_name)
    os.makedirs(processed_class_path, exist_ok=True)

    # Process each image
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)

        # Read image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping (not readable): {img_file}")
            continue

        try:
            # Resize
            resized_img = cv2.resize(img, imageSize)

            # Save to processed/folder
            save_path = os.path.join(processed_class_path, img_file)
            cv2.imwrite(save_path, resized_img)
        except Exception as e:
            print(f"Error processing {img_file}: {e}")

print("\n Processing completed. Images saved to 'dataset/processed'.")


Processing class: eraser
Processing class: glueStick
Processing class: mouse
Processing class: paperClip
Processing class: pen
Processing class: pencilBox
Processing class: pencilSharpener
Processing class: scissor
Processing class: stapler
Processing class: waterBottle

 Processing completed. Images saved to 'dataset/processed'.


It can be deduced that there is a class imbalance. Some classes may have fewer images than others, indicating that data augmentation or additional data collection is needed to balance the dataset. Classes with a low number of images  for e.g paper clips, are candidates for augmentation to increase training data and improve model performance.

Image augmentation



In [16]:
import os
import cv2
import random
import shutil

PROCESSED_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\processed"
AUGMENTED_DIR = r"C:\Users\23052\Desktop\C1- PDE 3802\RoboticsC1-PDE-3802\augmented"
TARGET_COUNT = 3000  # total images per class

# Create augmented folder if not exists
if not os.path.exists(AUGMENTED_DIR):
    os.makedirs(AUGMENTED_DIR)

# Augmentation functions
def rotate(image):
    angle = random.choice([15, -15, 30, -30])
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(image, M, (w, h))

def flip(image):
    return cv2.flip(image, 1)

def change_hue(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_shift = random.randint(-20, 20)
    # Convert to int to avoid overflow, apply shift, then modulo 180 and convert back to uint8
    hsv_h = hsv[:, :, 0].astype(int)
    hsv[:, :, 0] = ((hsv_h + hue_shift) % 180).astype('uint8')
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)




augmentations = [rotate, flip, change_hue]

# Process each class
for class_name in os.listdir(PROCESSED_DIR):
    class_path = os.path.join(PROCESSED_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"\nProcessing class: {class_name}")

    augmented_class_path = os.path.join(AUGMENTED_DIR, class_name)
    os.makedirs(augmented_class_path, exist_ok=True)

    # Step 1: Copy original images
    original_images = os.listdir(class_path)
    for img_file in original_images:
        shutil.copy(os.path.join(class_path, img_file),
                    os.path.join(augmented_class_path, img_file))

    # Step 2: Generate augmented images until reaching TARGET_COUNT
    count = len(original_images)
    while count < TARGET_COUNT:
        img_file = random.choice(original_images)
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        aug_func = random.choice(augmentations)
        aug_img = aug_func(img)
        new_filename = f"{os.path.splitext(img_file)[0]}_aug{count}.jpg"
        cv2.imwrite(os.path.join(augmented_class_path, new_filename), aug_img)
        count += 1

print("\n Augmentation complete. Originals and augmented images saved in 'augmented/'")



Processing class: eraser

Processing class: glueStick

Processing class: mouse

Processing class: paperClip

Processing class: pen

Processing class: pencilBox

Processing class: pencilSharpener

Processing class: scissor

Processing class: stapler

Processing class: waterBottle

 Augmentation complete. Originals and augmented images saved in 'augmented/'
